In [ ]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm
import torch
import matplotlib.pyplot as plt
import nn_utils as nn_ut

In [ ]:
# setting latex style for plots
plt.rcParams['text.usetex'] = True

In [ ]:
# defining the payoff function

K1 = 0.9
K2 = 1.2
def f(x):
    return torch.maximum(x - K1, torch.tensor(0.)) - torch.maximum(x - K2, torch.tensor(0.))

plt.plot(torch.arange(0.75,1.25,0.01), f(torch.arange(0.75,1.25,0.01)), label='payoff function')
plt.legend()
plt.show()

In [ ]:
# defining the reference measure

sigma = 0.20
t = .5
mu = torch.distributions.LogNormal(- 0.5 * sigma * sigma * t, sigma * math.sqrt(t))

samples = mu.sample([100000])
plt.hist(samples.detach().numpy(), 300, label='reference measure')
plt.legend()
plt.show()

In [ ]:
# cost functional
p = 3
h = 1. / 12
def cost(u):
    return h * torch.pow(torch.absolute(u) / math.sqrt(h), p)

### risk measure by neural network approximation

# fixing the seed
torch.manual_seed(29)

# defining the risk measure object
width = 20
depth = 4
sample_size = 100
risk_measure = nn_ut.MartRiskMeasure1d(f, cost, mu, torch.nn.ReLU, width, depth)

# otpimizer
optim = torch.optim.Adam(risk_measure.parameters(), lr=0.001)

# training cycle
train_hist = []
epochs = 20000
for i in range(epochs):
    optim.zero_grad()
    y = mu.sample([sample_size,1])
    risk = risk_measure(y)
    risk.backward()
    optim.step()
    train_hist.append(- float(risk.detach()))

In [ ]:
# plotting the training phase

roll_window = 100
train_roll = pd.Series(train_hist).rolling(roll_window).mean().dropna()

final_samples = mu.sample([100000,1])
expected_loss = torch.mean(f(final_samples))
plt.plot(1 + np.arange(roll_window, epochs + 1), train_roll, label=f'sup')
plt.axhline(float(expected_loss), color='green', label='expected payoff')
plt.legend()
plt.show()

In [ ]:
# results
risk_measure.eval()

mc_rm = -risk_measure(final_samples)
mc_err = 2.57 * risk_measure.mc_err
print(f"risk measure mc: {mc_rm:.3e}")
print(f"Mc interval: [{mc_rm - mc_err:.3e}, {mc_rm + mc_err:.3e}]")
print(f"Expected value: {expected_loss:.3e}")

In [ ]:
# plotting the results

x_plot = torch.arange(0.75, 1.25, 0.01)
net_plot = risk_measure.unet(x_plot.reshape(-1, 1))
u_plot, v_plot, p_plot = risk_measure._u_v_p(x_plot.reshape(-1, 1))
ccong = risk_measure.ccong(x_plot.reshape(-1, 1))
plt.plot(x_plot.detach(), f(x_plot).detach(), label='payoff function', color='blue')
plt.plot(x_plot.detach(), ccong.detach(), label=r'$C$-transform', color='green')
plt.legend()
plt.show()

fig, ax1 = plt.subplots()
ax1.plot(x_plot, u_plot.detach(), color='blue', label=r'$u^*$')
ax1.plot(x_plot, v_plot.detach(), color='green', label=r'$v^*$')
ax1.set_xlabel(r'$x$')
ax1.tick_params(axis='y', labelcolor='b')
ax2 = ax1.twinx()
ax2.plot(x_plot, p_plot.detach(), color='red', label=r'$p^*$')
ax2.set_ylim(0,1)
ax2.tick_params(axis='y', labelcolor='r')
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
legend = ax1.legend(lines + lines2, labels + labels2, loc='upper right')
plt.show()